In [1]:
import rasterio as rio
import numpy as np
import xarray as xr
import pygmt 

In [2]:
R=rio.open('./Documents/david_clip.tif')
Z=R.read(1)

In [4]:
fig = pygmt.Figure()
dem_data = Z.copy()
m,n = np.shape(Z)
I=[R.xy(i,0)[1] for i in range(m)]
J=[R.xy(0,i)[0] for i in range(n)]
#dem_data[dem_data<-200]=-200
m1,n1 = np.shape(dem_data)

#We aren't showing coordinates on in the map, so we can just use the raw coordinates from the weird projection the DEM is in.
#Replace with actual coords if using a LAT LON, etc.
dem_lon = J
dem_lat = I

#da is for the DEM
da = xr.DataArray(dem_data, coords=[('latitude', dem_lat,
                                        {'units': 'meters'}),
                                        ('longitude', dem_lon,
                                        {'units': 'meters'})]
                                )

#da2 is only for the hillshade
da2 = xr.DataArray(dem_data.copy(), coords=[('latitude', dem_lat,
                                        {'units': 'meters'}),
                                        ('longitude', dem_lon,
                                        {'units': 'meters'})]
                  )

dataset = da.to_dataset(name='dataarray')
dataset2 = da2.to_dataset(name='dataarray')

dataset2['dataarray'] *= 5 #Exagerate for the purpose of the hillshade
dgrid = pygmt.grdgradient(grid=dataset2.dataarray,radiance='m20/5+a.001+s1+d1', normalize=.5) #Hillshade is the dgrid


pygmt.makecpt(cmap ='dem4',series=[100, 1100],overrule_bg=True)
#fig.basemap(frame='True',region=[0, 1000, 300, 1000])
fig.grdimage(dataset.dataarray,cmap=True,frame="nwse",projection=f"X127/67c",shading=dgrid)
#%str(np.min(J)+1)}/{str(np.min(I)+1)}/{str(np.max(J)-np.min(J)-1)}/{str(np.max(I)-np.min(I)-1)}/{str(min(J))}12c")  #CenterLon/CenterLat/Lat1/Lat2/
#with pygmt.config(FONT='8p,black'):
    #fig.colorbar( frame="xa100f100+lElevation (m)")


-Cdem4 -M -T100/1100


In [ ]:
fig.show()

In [6]:
fig.savefig('./Documents/eger2.png')

In [17]:
np.shape(Z)

(8987, 20829)